In [1]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Importar librerías generales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Cargar el dataset preprocesado
dataset = pd.read_csv("/content/drive/MyDrive/disaster_preprocessed.csv", encoding='utf-8')
dataset.head()

,InformationType,event,TweetID,TweetText,location,year,Informativeness_label,ProcessedText,ProcessedText_length
0,not related or not informative,landslides,'577702417766060032',HAPPY SAINT PATRIC'S DAAAAAAAAAAAAY ÃÂ _ÃÂ0...,Worldwide,unknown,not related or not informative,happy saint patric ' s daaaaaaaaaaaay,6
1,caution and advice,hurricane,7.60519e+16,@7News: New tornado warning for storm in Sprin...,Missouri,2011,informative,new tornado warning for storm in springfield /...,47
2,not related or not informative,landslides,'573782308551004160',"""@irishcraicc: ÃÂ __8ÃÂ¤ÃÂÃÂÃÂ¥ÃÂÃÂ...",Worldwide,unknown,not related or not informative,""" 8 quote and rt this for free follows a # kca...",20
3,donations and volunteering,hurricane,7.3445996344e+16,As organizations and first responders take adv...,Missouri,2011,informative,as organizations and first responders take adv...,40
4,not related or not informative,hurricane,'541686284302188546',PS 63:3-4 Ur love Lord s better than life my l...,Philipinnes,2014,not related or not informative,ps 63 : 3 - 4 ur love lord s better than life ...,42


In [4]:
from sklearn.model_selection import train_test_split

# División inicial: Training (80%), Validation (5000), Test (5000)
train_df, temp_df = train_test_split(dataset, test_size=10000, random_state=42, stratify=dataset['InformationType'])

# Dividir temp_df en validación y test (50% cada uno)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['InformationType'])

print(f"Tamaño de training: {len(train_df)}, validación: {len(val_df)}, prueba: {len(test_df)}")

Tamaño de training: 64346, validación: 5000, prueba: 5000


In [5]:
# Importar el tokenizer de BERT
from transformers import BertTokenizer

# Tokenizer preentrenado
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Codificación de etiquetas usando la columna "InformationType"
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df['InformationType'])
val_labels = label_encoder.transform(val_df['InformationType'])
test_labels = label_encoder.transform(test_df['InformationType'])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader

# Definir la clase del dataset compatible con PyTorch
class DisasterDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encoding = self.tokenizer(
            self.texts[idx],
            padding='max_length',
            truncation=True,
            max_length=128,
            return_tensors='pt'
        )
        # Squeeze para quitar dimensiones innecesarias
        return {key: val.squeeze() for key, val in encoding.items()}, torch.tensor(self.labels[idx])

# Crear datasets y DataLoaders
train_dataset = DisasterDataset(list(train_df['ProcessedText']), train_labels, tokenizer)
val_dataset = DisasterDataset(list(val_df['ProcessedText']), val_labels, tokenizer)
test_dataset = DisasterDataset(list(test_df['ProcessedText']), test_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


In [7]:
from transformers import BertForSequenceClassification

# Definir el modelo con dropout (ajustando el número de etiquetas)
information_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_encoder.classes_),
    hidden_dropout_prob=0.3
)

# Enviar el modelo a GPU (si está disponible)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
information_model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
from sklearn.metrics import classification_report

def evaluate(loader, model):
    """Evalúa el modelo en un conjunto de datos y retorna la pérdida y el reporte de clasificación."""
    model.eval()
    predictions, true_labels = [], []
    total_loss = 0

    with torch.no_grad():
        for batch in loader:
            batch_inputs, batch_labels = batch
            batch_inputs = {key: val.to(device) for key, val in batch_inputs.items()}
            batch_labels = batch_labels.to(device)

            outputs = model(**batch_inputs)
            loss = loss_fn(outputs.logits, batch_labels)
            total_loss += loss.item()

            predictions.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            true_labels.extend(batch_labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    report = classification_report(true_labels, predictions, target_names=label_encoder.classes_, digits=3)
    return avg_loss, report


In [9]:
from sklearn.model_selection import StratifiedKFold
from transformers import AdamW, get_scheduler

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
fold_results = []

for fold, (train_index, val_index) in enumerate(skf.split(train_df['ProcessedText'], train_labels)):
    print(f"\n🔄 Fold {fold + 1}")

    # División de los datos para este fold
    train_texts = train_df.iloc[train_index]['ProcessedText']
    val_texts = train_df.iloc[val_index]['ProcessedText']
    train_labels_fold = train_labels[train_index]
    val_labels_fold = train_labels[val_index]

    # Crear datasets y dataloaders para el fold
    train_dataset_fold = DisasterDataset(list(train_texts), train_labels_fold, tokenizer)
    val_dataset_fold = DisasterDataset(list(val_texts), val_labels_fold, tokenizer)

    train_loader_fold = DataLoader(train_dataset_fold, batch_size=32, shuffle=True)
    val_loader_fold = DataLoader(val_dataset_fold, batch_size=32)

    # Inicializar el modelo para el fold
    information_model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=len(label_encoder.classes_),
        hidden_dropout_prob=0.3
    )
    information_model.to(device)

    # Definir optimizador, función de pérdida y scheduler
    optimizer = AdamW(information_model.parameters(), lr=2e-5, eps=1e-8)
    loss_fn = torch.nn.CrossEntropyLoss()
    scheduler = get_scheduler(
        'linear',
        optimizer=optimizer,
        num_warmup_steps=0,
        num_training_steps=len(train_loader_fold) * 3  # 3 epochs
    )

    # Entrenamiento del fold
    for epoch in range(3):
        information_model.train()
        total_loss = 0

        for batch in train_loader_fold:
            batch_inputs, batch_labels = batch
            batch_inputs = {key: val.to(device) for key, val in batch_inputs.items()}
            batch_labels = batch_labels.to(device)

            outputs = information_model(**batch_inputs, labels=batch_labels)
            loss = outputs.loss
            total_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            scheduler.step()

        # Evaluación en el set de validación
        val_loss, val_report = evaluate(val_loader_fold, information_model)
        print(f"Epoch {epoch + 1} - Training Loss: {total_loss / len(train_loader_fold):.4f}, Validation Loss: {val_loss:.4f}")
        print(val_report)

    fold_results.append(val_loss)

print(f"\n📊 Cross-validation average loss: {np.mean(fold_results):.4f} ± {np.std(fold_results):.4f}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🔄 Fold 1


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 - Training Loss: 1.0247, Validation Loss: 0.8790
                                     precision    recall  f1-score   support

               affected individuals      0.681     0.814     0.742      2311
                 caution and advice      0.530     0.493     0.511       914
         donations and volunteering      0.621     0.851     0.718      2575
infrastructure and utilities damage      0.575     0.587     0.581      1316
     not related or not informative      0.826     0.665     0.737      7439
           other useful information      0.617     0.645     0.631      5446
               sympathy and support      0.711     0.636     0.672      1448

                           accuracy                          0.684     21449
                          macro avg      0.652     0.670     0.656     21449
                       weighted avg      0.697     0.684     0.685     21449

Epoch 2 - Training Loss: 0.8046, Validation Loss: 0.8416
                                    

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 - Training Loss: 1.0273, Validation Loss: 0.8960
                                     precision    recall  f1-score   support

               affected individuals      0.656     0.823     0.730      2310
                 caution and advice      0.510     0.475     0.492       915
         donations and volunteering      0.636     0.848     0.727      2575
infrastructure and utilities damage      0.562     0.620     0.590      1315
     not related or not informative      0.818     0.691     0.749      7439
           other useful information      0.632     0.597     0.614      5446
               sympathy and support      0.681     0.648     0.664      1449

                           accuracy                          0.684     21449
                          macro avg      0.642     0.672     0.652     21449
                       weighted avg      0.693     0.684     0.683     21449

Epoch 2 - Training Loss: 0.8056, Validation Loss: 0.8255
                                    

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 - Training Loss: 1.0421, Validation Loss: 0.8740
                                     precision    recall  f1-score   support

               affected individuals      0.710     0.807     0.755      2311
                 caution and advice      0.539     0.396     0.456       915
         donations and volunteering      0.703     0.783     0.741      2574
infrastructure and utilities damage      0.652     0.509     0.572      1315
     not related or not informative      0.812     0.665     0.731      7439
           other useful information      0.592     0.679     0.633      5446
               sympathy and support      0.535     0.711     0.611      1448

                           accuracy                          0.680     21448
                          macro avg      0.649     0.650     0.643     21448
                       weighted avg      0.692     0.680     0.680     21448

Epoch 2 - Training Loss: 0.8103, Validation Loss: 0.8225
                                    

In [12]:
test_loss, test_report = evaluate(test_loader, information_model)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Classification Report:\n{test_report}")

Test Loss: 0.8323
Test Classification Report:
                                     precision    recall  f1-score   support

               affected individuals      0.718     0.805     0.759       538
                 caution and advice      0.523     0.535     0.529       213
         donations and volunteering      0.661     0.820     0.732       601
infrastructure and utilities damage      0.556     0.616     0.584       307
     not related or not informative      0.819     0.706     0.759      1734
           other useful information      0.623     0.620     0.622      1270
               sympathy and support      0.724     0.715     0.719       337

                           accuracy                          0.697      5000
                          macro avg      0.661     0.688     0.672      5000
                       weighted avg      0.704     0.697     0.698      5000



In [13]:
# Guardar el modelo y el tokenizer
information_model.save_pretrained('bert_information_classifier')
tokenizer.save_pretrained('bert_information_classifier')
print("Modelo y tokenizer guardados en 'bert_information_classifier'")

# Guardar el label encoder
import joblib
joblib.dump(label_encoder, 'label_encoder.pkl')

# Comprimir el modelo guardado
!zip -r bert_information_classifier.zip bert_information_classifier

Modelo y tokenizer guardados en 'bert_information_classifier'
updating: bert_information_classifier/ (stored 0%)
updating: bert_information_classifier/config.json (deflated 55%)
updating: bert_information_classifier/tokenizer_config.json (deflated 75%)
updating: bert_information_classifier/model.safetensors (deflated 7%)
updating: bert_information_classifier/special_tokens_map.json (deflated 42%)
updating: bert_information_classifier/vocab.txt (deflated 53%)
